In [1]:
import random
import math


def generate(Board, N):
    for i in range(0, N):
        row = []
        for j in range(0, N):
            row.append(0)
        Board.append(row)

    for i in range(0, N):
        rand_row = random.randint(0, (N-1))
        Board[rand_row][i] = 1

    return Board

In [2]:
def Objective_Function(Board, N):
    count = 0
    for i in range(0, N):
        for j in range(0, N):
            if Board[i][j] == 1:

                # Attacks in the same row
                for k in range(0, N):
                    if Board[i][k] == 1 and k != j:
                        count += 1

                # Attacks in the same column
                for k in range(0, N):
                    if Board[k][j] == 1 and k != i:
                        count += 1

                # Diagonal Attacks up the diagonal (R -> L)
                current_row_1 = i
                current_col_1 = j
                while current_row_1 > 0 and current_col_1 < (N-1):
                    if Board[current_row_1-1][current_col_1+1] == 1:
                        count += 1
                    current_row_1 -= 1
                    current_col_1 += 1

                #Diagonal Attacks down the diagonal (R -> L)
                current_row_2 = i
                current_col_2 = j
                while current_row_2 < (N-1) and current_col_2 > 0:
                    if Board[current_row_2+1][current_col_2-1] == 1:
                        count += 1
                    current_col_2 -= 1
                    current_row_2 += 1


                #Diagonal Attacks up the diagonal (L -> R)
                current_row_L = i
                current_col_L = j
                while current_row_L > 0 and current_col_L > 0:
                    if Board[current_row_L-1][current_col_L-1] == 1:
                        count += 1
                    current_row_L -= 1
                    current_col_L -= 1

                #Diagonal Attacks down the diagonal (L -> R)
                current_row_L2 = i
                current_col_L2 = j
                while current_row_L2 < (N-1) and current_col_L2 < (N-1):
                    if Board[current_row_L2+1][current_col_L2+1] == 1:
                        count += 1
                    current_row_L2 += 1
                    current_col_L2 += 1



    return count




In [3]:
def getNeighbors(Board, N):
    boardsList = []
    # Loop over the entire Board
    for i in range(N):
        for j in range(N):
            # newBoards represent the next 2 possible neighbor states
            # Initially, they will be equal to the Board
            newBoard1 = [row[:] for row in Board]
            newBoard2 = [row[:] for row in Board]
            if Board[i][j] == 1:

                # if not on the top row and not on the bottom row allow
                # up and down moves
                if i != 0 and i != (N-1):
                    # Down neighbor
                    newBoard1[i][j] = 0
                    newBoard1[i+1][j] = 1
                    # Up neighbor
                    newBoard2[i][j] = 0
                    newBoard2[i-1][j] = 1
                    # Add to the boards list
                    boardsList.append(newBoard1)
                    boardsList.append(newBoard2)
                # If on the top row allow only down move
                elif i == 0:
                    newBoard2[i][j] = 0
                    newBoard2[i+1][j] = 1
                    boardsList.append(newBoard2)
                # If on the last row allow only up move
                elif i == N-1:
                    newBoard1[i][j] = 0
                    newBoard1[i-1][j] = 1
                    boardsList.append(newBoard1)
    return boardsList

# Print Board Function - parameter: The Board List
def printBoard(Board):
    print('\n'.join([''.join(['{:4}'.format(x) for x in row])
                     for row in Board]))




In [4]:
# Hill Climbing Algorithm
def hillClimbing(Board, N):
    # Get the number of attacks on the original board
    count = Objective_Function(Board, N)
    # If the attacks = 0, return from the function
    if count == 0:
        return Board
    else:
        # Updated State is the next immediate state/neighbor of the Board
        # that will be selected later
        updatedState = [row[:] for row in Board]
        updatedCount = count
        iteration = 0
        while True:
            # If the updated Count is the same or greater as in the previous iteration
            # that means we have reached a plateau of local minima
            if count == 0 or (updatedCount >= count and iteration > 0):
                break
            # Get a list of neighbor boards
            boards_list = getNeighbors(updatedState, N)
            updatedCount = count
            # Loop over the list of neighbors
            for board in boards_list:
                # Evaluate the objective function for the board
                attackCount = Objective_Function(board, N)
                # get the board with the minimum attack count
                if attackCount < count:
                    # update to a new state
                    updatedState = [row[:] for row in board]
                    count = attackCount
            iteration += 1

    return updatedState




In [5]:
def simulatedAnnealing(Board, N):
    solution = [row[:] for row in Board]
    # Maximum value for t
    t = 10000
    boards_list = []
    # While t is almost 0
    while t > 1:
        # Get the number of attacks possible from the solution
        soln_cost = Objective_Function(solution, N)
        # Get a list of all the neighbors
        boards_list = getNeighbors(solution, N)
        # Select one neighbor at random
        random_board = random.choice(boards_list)
        # Calculate the number of attacks of the neighbor
        neigh_cost = Objective_Function(random_board, N)
        # If number of attacks of the neighbor are less than the current solutions attacks
        if neigh_cost < soln_cost:
            # update solution
            solution = [row[:] for row in random_board]
            # Get new attacks cost
            new_soln_cost = Objective_Function(solution, N)
            if new_soln_cost == 0:
                break
        else:
            # calculate the change in cost
            cost_increase = (neigh_cost-soln_cost)
            # calculate p
            p = math.exp((-1*cost_increase)/float(t))
            if cost_increase < 0 or random.uniform(0, 1) < p:
                #  0.01 decay factor
                t = t-0.01
                solution = [row[:] for row in random_board]
    # print(t)

    return solution




In [6]:
if __name__ == '__main__':
    # Create a Board 2D Array
    Board = []
    hillBoard = []
    simulatedBoard = []
    # Init the board with random values
    N = 8
    Board = generate(Board, N)

    # Print the board
    print("Randomly Generated Board")
    initBoardAttacks = Objective_Function(Board, N)
    print("Number of Attacks: ", initBoardAttacks)
    printBoard(Board)

    print("\nHill Climbing Board")
    hillBoard = hillClimbing(Board, N)
    hillBoardAttacks = Objective_Function(hillBoard, N)
    print("Number of Attacks: ", hillBoardAttacks)
    printBoard(hillBoard)

    print("\nSimulated Annealing Board")
    print("***** Please wait, the answer does get computed, but often takes time to generate the board *****")
    simulatedBoard = simulatedAnnealing(Board, N)
    simulatedAttacks = Objective_Function(simulatedBoard, N)
    print("Number of Attacks: ", simulatedAttacks)
    printBoard(simulatedBoard)



Randomly Generated Board
Number of Attacks:  12
   0   0   0   0   0   0   0   0
   0   1   0   0   0   1   0   0
   0   0   0   0   0   0   0   1
   1   0   0   1   0   0   0   0
   0   0   0   0   0   0   0   0
   0   0   0   0   1   0   0   0
   0   0   1   0   0   0   0   0
   0   0   0   0   0   0   1   0

Hill Climbing Board
Number of Attacks:  8
   0   1   0   0   0   0   0   0
   0   0   0   0   0   1   0   0
   0   0   0   0   0   0   0   1
   1   0   0   1   0   0   0   0
   0   0   0   0   0   0   0   0
   0   0   0   0   1   0   0   0
   0   0   1   0   0   0   0   0
   0   0   0   0   0   0   1   0

Simulated Annealing Board
***** Please wait, the answer does get computed, but often takes time to generate the board *****
Number of Attacks:  0
   0   0   0   0   0   0   1   0
   0   0   1   0   0   0   0   0
   1   0   0   0   0   0   0   0
   0   0   0   0   0   1   0   0
   0   0   0   0   0   0   0   1
   0   0   0   0   1   0   0   0
   0   1   0   0   0   0   0   0
   